# Prediction with .h5 file

### Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import numpy as np

import tools

# Root CSV files directory
dirname = "./prediction_data/"

# Frame count
frames = 100

In [ ]:
model = tf.keras.models.load_model('sign_lang_recognition.h5')
model.summary()

#### Load the file to predict

In [2]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train=np.array(features)

tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([labels])[0]
y_train = to_categorical(encoded_train)
print(y_train)

{'deutschland': 1}
[[0. 1.]
 [0. 1.]]


In [8]:
print(x_train)

[[[0.507547 0.382073 0.213684 ... 0.       0.       0.      ]
  [0.50686  0.383572 0.274671 ... 0.834884 0.737334 0.852904]
  [0.505028 0.391949 0.284391 ... 0.751541 0.714473 0.775634]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]

 [[0.505633 0.400101 0.451716 ... 0.       0.       0.      ]
  [0.502219 0.403942 0.451105 ... 0.       0.       0.      ]
  [0.505479 0.400694 0.448652 ... 0.       0.       0.      ]
  ...
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]
  [0.       0.       0.       ... 0.       0.       0.      ]]]


In [ ]:
y_pred = model.predict(x_train)

In [ ]:
predictions = np.array([np.argmax(pred) for pred in y_pred])
print(predictions)

In [7]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
# Test model on random input data.
interpreter.set_tensor(input_details[0]['index'], x_train)

#interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
#output_data = interpreter.get_tensor(output_details[0]['index'])
#print(output_data)

[{'name': 'lstm_input', 'index': 0, 'shape': array([  1, 100,  86], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


ValueError: Cannot set tensor: Got value of type NOTYPE but expected type FLOAT32 for input 0, name: lstm_input 